In [51]:
import pandas as pd
from pandas.api.types import is_numeric_dtype
import numpy as np
from ast import literal_eval

import warnings
warnings.filterwarnings('ignore')

In [2]:
%%time

data = pd.read_csv('data/autok_final.csv.gz', compression = 'gzip')

Wall time: 3.56 s


### Correct data types

#### Clean column names

In [3]:
data.columns = [i.replace(':', '').strip() for i in data.columns]

#### Check missing values

In [4]:
NA_count = data.isnull().sum() / data.shape[0]
NA_count[NA_count > 1/3]

Extrákkal növelt ár       0.906094
Akciós ár                 0.912635
Akció feltételei          0.916953
Átvehető                  0.891906
Modellcsoport             0.728347
Fordulási kör átmérője    0.384454
Oktánszám                 0.522199
Garancia                  0.945147
Hasmagasság               0.816818
Első terepszög            0.900866
Hátsó terepszög           0.900775
Rámpaszög                 0.926797
Gázlómélység              0.958047
Alaptípus ára             0.935848
dtype: float64

Some of the price features will be used later, but the car features with most NaN can be dropped

In [5]:
drop_due_to_NA = ['Oktánszám', 'Fordulási kör átmérője', 'Hasmagasság', 'Gázlómélység', 'Garancia',
                  'Átvehető', 'Első terepszög', 'Hátsó terepszög', 'Rámpaszög', 'Akció feltételei']
data.drop(drop_due_to_NA, 1, inplace = True)

#### Check dtypes - clean columns of unnecessary strings (i.e: 50kW --> 50, 5 doors --> 5)

In [6]:
print(data.shape)
data = data[data['Vételár'] != 'Ár nélkül']
print(data.shape)

(76896, 42)
(76865, 42)


In [7]:
%%time

# has a comma between numbers, need to replace it with dot and make it a number
data['Gyorsulás 0-ról 100 km/h-ra'] = data['Gyorsulás 0-ról 100 km/h-ra'].str.replace('s', '').str.strip().str.replace(',', '.').astype(float)

# others dont have commas
numeric_cols = ['Vételár', 'Vételár EUR', 'Extrákkal növelt ár', 'Akciós ár', 'Alaptípus ára',
                'Kilométeróra állása', 'Szállítható szem. száma', 'Ajtók száma', 'Saját tömeg', 
                'Teljes tömeg', 'Csomagtartó', 'Hengerűrtartalom', 'Szélesség', 'Magasság', 'Hosszúság', 'Tengelytáv', 
                'Üzemanyagtank', 'Hasznos teher', 'CO2-kibocsátás', 'Végsebesség']

# fogyasztas will need to be treated differently as there are two numbers there (5.6l/100km)

# for i in numeric_cols:
#     if i in data.columns:
#         if is_numeric_dtype(data[i]) == False:
#             print(data[i].str.replace(r'\d+', '').replace('', np.nan).value_counts(dropna = False))

# vetelar (HUF) has a couple of EUR values, treat them
data.loc[data['Vételár'].fillna('').str.contains('€'), 'Vételár'] = np.nan

for i in numeric_cols:
    if i in data.columns:
        if is_numeric_dtype(data[i]) == False:
            data[i] = data[i].str.replace(r'\D+', '').replace('', np.nan).astype(float)

Wall time: 2.65 s


Remainind columns to be treated:
- Teljesítmény (55kW, 75LE --> split into two)
- Fogyasztások (városi, országúti, vegyes) (8,6 l/100km	--> 8.6)

In [8]:
data[['Teljesítmény kW', 'Teljesítmény LE']] = data['Teljesítmény'].str.split(',', expand = True)
data.drop('Teljesítmény', 1, inplace = True)

In [9]:
teljesitmeny = ['Teljesítmény kW', 'Teljesítmény LE']

for i in teljesitmeny:
    if i in data.columns:
        if is_numeric_dtype(data[i]) == False:
            data[i] = data[i].str.replace(r'\D+', '').replace('', np.nan).astype(float)

In [10]:
fogyasztasok = data.filter(like = 'fogyasztás').columns

for i in fogyasztasok:
    if i in data.columns:
        data[i] = data[i].str.replace('l/100km', '').str.strip().str.replace(',', '.').astype(float)

#### Convert remaining string columns that should be numeric to floats
Évjárat:
- delete everything between paranthesis
- delete everything after first / sign
- convert string year to int year

Műszaki vizsga érvényes
- delete everything after first / sign
- convert string year to int year (Int64 due to NaNs)

In [11]:
data['Évjárat'] = data['Évjárat'].str.replace(r'\(.*\)', '').str.replace('(/).*', '').str.strip().astype(int)

In [12]:
data['Műszaki vizsga érvényes'] = data['Műszaki vizsga érvényes'].str.replace('(/).*', '').replace('', np.nan).str.strip().astype(float)

### Check categorical features - normalize them, group them

In [13]:
data.select_dtypes('O').describe().T

,count,unique,top,freq
Állapot,76865,18,Normál,28003
Kivitel,76865,15,Ferdehátú,24108
Üzemanyag,76319,10,Benzin,40937
Hajtás,71319,5,Első kerék,51770
Sebességváltó fajtája,72332,49,Manuális (5 fokozatú),29050
Okmányok jellege,76854,7,Érvényes magyar okmányokkal,65511
URL,76865,76865,https://www.hasznaltauto.hu/szemelyauto/suzuki...,1
Hirdetés név,76865,56890,BMW 320d,76
Felszereltség,71933,65900,"{'Beltér': ['bőr belső', 'ülésmagasság állítás...",144
Kategória,68112,9,Alsó középkategória,19861


In [14]:
%%time

data.loc[data['Állapot'].str.lower().str.contains('sérült', na = False), 'Állapot'] = 'Sérült'
data.loc[data['Állapot'].str.lower().str.contains('hibás', na = False), 'Állapot'] = 'Sérült'
data.loc[data['Állapot'].str.lower().str.contains('hiány', na = False), 'Állapot'] = 'Sérült'

data = data[~data['Kivitel'].isin(['Buggy', 'Mopedautó', 'Egyéb', 'Sport'])]
data.loc[data['Kivitel'].str.lower().str.contains('terep', na = False), 'Kivitel'] = 'Terepjáró'
data.loc[data['Kivitel'].str.lower().str.contains('pickup', na = False), 'Kivitel'] = 'Terepjáró'
data.loc[data['Kivitel'].str.lower().str.contains('lépcsőshátú', na = False), 'Kivitel'] = 'Coupe'

data = data[~data['Üzemanyag'].isin(['Etanol', 'CNG', 'LPG'])]
data.loc[data['Üzemanyag'].str.lower().str.contains('hibrid', na = False), 'Üzemanyag'] = 'Hibrid'
data.loc[data['Üzemanyag'].str.lower().str.contains('benzin', na = False), 'Üzemanyag'] = 'Benzin'

data.loc[data['Hajtás'].str.lower().str.contains('össz', na = False), 'Hajtás'] = 'Összkerék'

data.loc[data['Okmányok jellege'].str.lower().str.contains('érvényes', na = False), 'Okmányok jellege'] = 'Érvényes'
data.loc[data['Okmányok jellege'].str.lower().str.contains('lejárt', na = False), 'Okmányok jellege'] = 'Lejárt'
data.loc[data['Okmányok jellege'].str.lower().str.contains('kivont', na = False), 'Okmányok jellege'] = 'Kivont'

data = data[data['Klíma fajtája'] != 'Hőszivattyús klíma']
data.loc[data['Klíma fajtája'].str.lower().str.contains('digitális', na = False), 'Klíma fajtája'] = 'Digitális'
data.loc[data['Klíma fajtája'].str.lower().str.contains('manuális', na = False), 'Klíma fajtája'] = 'Manuális'
data.loc[data['Klíma fajtája'].str.lower().str.contains('automata', na = False), 'Klíma fajtája'] = 'Automata'

data['Szín - metál'] = data['Szín'].apply(lambda x: 1 if 'metál' in str(x).lower() else 0)
data['Szín'] = data['Szín'].str.replace(r'\(.*\)', '').str.strip().replace('', np.nan)
data.loc[data['Szín'].str.lower().str.contains('szürke', na = False), 'Szín'] = 'Szürke'
data.loc[data['Szín'].str.lower().str.contains('kék', na = False), 'Szín'] = 'Kék'
data.loc[data['Szín'].str.lower().str.contains('narancs', na = False), 'Szín'] = 'Narancssárga'
data.loc[data['Szín'].str.lower().str.contains('sárga', na = False), 'Szín'] = 'Sárga'
data.loc[data['Szín'].str.lower().str.contains('okker', na = False), 'Szín'] = 'Sárga'
data.loc[data['Szín'].str.lower().str.contains('fehér', na = False), 'Szín'] = 'Fehér'
data.loc[data['Szín'].str.lower().str.contains('barna', na = False), 'Szín'] = 'Barna'
data.loc[data['Szín'].str.lower().str.contains('zöld', na = False), 'Szín'] = 'Zöld'
data.loc[data['Szín'].str.lower().str.contains('türkiz', na = False), 'Szín'] = 'Kék'
top_szinek = ['Fekete', 'Szürke', 'Fehér', 'Ezüst', 'Kék', 'Piros', 'Barna', 'Zöld', 'Bordó', 'Sárga']
data.loc[~data['Szín'].isin(top_szinek) & data['Szín'].notnull(), 'Szín'] = 'Egyéb'

data.loc[data['Sebességváltó fajtája'].str.lower().str.contains('automata', na = False), 'Sebességváltó fajtája'] = 'Automata'
data.loc[data['Sebességváltó fajtája'].str.lower().str.contains('szekvenciális', na = False), 'Sebességváltó fajtája'] = 'Automata'
data.loc[data['Sebességváltó fajtája'].str.lower().str.contains('manuális', na = False), 'Sebességváltó fajtája'] = 'Manuális'
data.loc[data['Sebességváltó fajtája'].str.lower().str.contains('tiptronic', na = False), 'Sebességváltó fajtája'] = 'Automata'

Wall time: 2.84 s


In [15]:
marka_count = data['Márka'].value_counts()
drop_brands = marka_count[marka_count < 300].index.tolist()

drop_brands = ['SMART', 'JAGUAR', 'LADA', 'CHRYSLER', 'MERCEDES-AMG', 'SSANGYONG', 'DAEWOO', 'SUBARU', 'TESLA', 'LANCIA', 'DODGE', 'INFINITI', 'SAAB', 'DAIHATSU', 'TRABANT', 'MASERATI', 'ROVER', 'CADILLAC', 'ABARTH', 'WARTBURG', 'CUPRA', 'LINCOLN', 'FERRARI', 'POLSKI FIAT', 'BENTLEY', 'VERSENYAUTÓ', 'ISUZU', 'MG', 'PONTIAC', 'HUMMER', 'ASTON MARTIN', 'GAZ', 'UAZ', 'ROLLS-ROYCE', 'MARUTI', 'DS', 'BUICK', 'TATA', 'BLUECAR', 'GMC', 'ZASTAVA', 'VOLGA', 'MOSZKVICS', 'IVECO', 'ACURA', 'OLDSMOBILE', 'ALPINA', 'MAHINDRA', 'LAMBORGHINI', 'FORD (USA)', 'NSU', 'MATRA', 'DATSUN', 'FISKER', 'YUGO', 'LOTUS', 'BARKAS', 'POLESTAR', 'AIXAM', 'GAC GONOW', 'GREAT WALL', 'DAIMLER', 'INNOCENTI', 'PROTON', 'DKW', 'PLYMOUTH', 'LUAZ', 'AUTOBIANCHI', 'TAZZARI', 'DUTTON', 'ARO', 'BERTONE', 'MERCURY', 'WARSZAWA', 'BRILLIANCE (BMW)', 'ADLER', 'WOLSELEY', 'ASIA', 'REPLIKA', 'POLARIS', 'TRIUMPH', 'HUDSON', 'FORD (AUSTRALIA)', 'PACKARD', 'GEO', 'NASH', 'MORGAN', 'SVW (VOLKSWAGEN)', 'ZAPOROZSEC', 'LIGIER', 'EXCALIBUR', 'EGYEDI', 'MERCEDES-MAYBACH', 'GOGGOMOBIL', 'BORGWARD', 'TALBOT', 'BRILLIANCE (ZHONGHUA)', 'AUSTIN']
data = data[~data['Márka'].isin(drop_brands)]

Set NaN drop rules before consolidating Models

In [16]:
data['Műszaki vizsga érvényes'] = data['Műszaki vizsga érvényes'].apply(lambda x: 'Érvényes' if x >= 2021 else 'Lejárt / Nincs adat')

data['Ajtók száma'].fillna('Nincs adat', inplace = True)
data['Ajtók száma'] = data['Ajtók száma'].astype(str)

data = data[(data['Szállítható szem. száma'] < 20) | (data['Szállítható szem. száma'].isnull())]
data['Szállítható szem. száma'] = data['Szállítható szem. száma'].apply(lambda x: '<5' if x < 5 else ('>5' if x > 5 else ('=5' if x == 5 else 'NA')))

Márka is too broad, modell is too low-level. Using Modellcsoport

In [17]:
%%time

data['Modellcsoport modellezéshez'] = data['Modellcsoport'].copy()

modell_count = data['Modell'].value_counts()
modell_count = modell_count[modell_count >= 150].index.tolist()
modell_count = ['FOCUS', 'OCTAVIA', 'A4', 'SWIFT', 'A6', 'MEGANE', 'MONDEO', 'ASTRA H', 'FIESTA', '320', "CEE'D", 'YARIS', 'A3', 'CLIO', 'FABIA', 'I30', 'CIVIC', '6', 'ASTRA J', 'QASHQAI', '500', 'COROLLA', 'PASSAT', 'CORSA D', 'ASTRA G', '3', 'INSIGNIA', 'X5', '530', 'SUPERB', 'KUGA', 'JAZZ', 'C-MAX', 'POLO', 'ASTRA K', 'AVENSIS', '206', '318', 'GOLF VII', 'VITARA', 'PASSAT VIII', 'PUNTO', '520', 'AURIS', 'GOLF V', 'LEON', 'PUMA', 'SCENIC', 'X3', 'TRANSIT', 'IBIZA', 'E 220', 'MICRA', 'IGNIS', 'GOLF VI', 'SPORTAGE', '308', 'TOURAN', 'SX4', 'TIPO', 'C3', 'CORSA F', 'X-TRAIL', 'A5', 'C 220', 'Q7', 'JUKE', 'RAV 4', 'S-MAX', 'ZAFIRA B', 'LAGUNA', 'X6', '307', 'GOLF IV', 'TIGUAN', '207', 'I3', '330', '2', 'MERIVA A', 'DUSTER', 'XC60', 'GALAXY', 'I20', 'Q5', 'ACCORD', '525', 'GOLF', 'LEAF', 'XSARA PICASSO', 'TUCSON', 'C4', 'SANDERO', 'MERIVA', '116', 'C 200', 'CORSA C', 'CROSSLAND', 'MUSTANG', 'CR-V', 'SX4 S-CROSS', 'TOUAREG', 'C5', 'OUTLANDER', 'GRANDE PUNTO', 'ALTEA', 'PASSAT VI', 'A8', '316', 'COLT', '5008', '3008', 'E 200', 'CAYENNE', '5', 'XC90', 'SORENTO', 'ASTRA', 'GOLF PLUS', 'ASTRA F', 'PANDA', 'TWINGO', 'RIO', '3-AS SOROZAT', '159', 'VECTRA C', 'ECOSPORT', '208', 'AYGO', 'ESPACE', 'MOKKA', 'PASSAT V', 'X1', 'S60', 'CORSA', 'C 180', 'POLO IV']
data.loc[(data['Modellcsoport'].isnull()) & (data['Modell'].isin(modell_count)), 'Modellcsoport modellezéshez'] = data.loc[(data['Modellcsoport'].isnull()) & (data['Modell'].isin(modell_count)), 'Modell']

modellcsoport_count = data['Modellcsoport modellezéshez'].value_counts()
modellcsoport_count = modellcsoport_count[modellcsoport_count >= 150].index.tolist()
modellcsoport_count = ['ASTRA', '3-AS SOROZAT', 'FOCUS', 'GOLF', 'PASSAT', 'X SOROZAT', '5-ÖS SOROZAT', 'OCTAVIA', 'CORSA', 'E-OSZTÁLY', 'A4', 'C-OSZTÁLY', 'SWIFT', 'A6', 'MEGANE', 'MONDEO', "CEE'D", 'FIESTA', 'POLO', 'YARIS', 'A3', 'COROLLA', 'CLIO', 'FABIA', 'ZAFIRA', 'I30', 'MERIVA', 'A-OSZTÁLY', 'CIVIC', '6', 'QASHQAI', 'PUNTO', '1-ES SOROZAT', '500', '3', 'SCENIC', 'INSIGNIA', 'SUPERB', 'KUGA', 'JAZZ', 'AVENSIS', 'VITARA', 'S-OSZTÁLY', 'C-MAX', '206', '7-ES SOROZAT', 'AURIS', 'LEON', 'PUMA', 'B-OSZTÁLY', 'MICRA', 'TRANSIT', 'IBIZA', 'A5', 'C4 PICASSO', 'IGNIS', 'SPORTAGE', 'XSARA', '308', 'TOURAN', 'SX4', 'TIPO', 'VECTRA', 'C3', 'X-TRAIL', 'I SOROZAT', 'Q7', 'JUKE', 'TIGUAN', 'RAV 4', 'S-MAX', 'LAGUNA', 'MINI', 'JETTA', '307', '207', '2', 'DUSTER', 'XC60', 'I20', 'GALAXY', 'ACCORD', 'Q5', 'ML-OSZTÁLY', 'LEAF', 'GLE-OSZTÁLY', 'TUCSON', 'SANDERO', 'C4', 'CROSSLAND', 'MUSTANG', 'CR-V', 'SX4 S-CROSS', 'ESPACE', 'TOUAREG', 'C5', 'OUTLANDER', 'ALTEA', 'A8', 'GLC-OSZTÁLY', 'M SOROZAT', 'COLT', '5008', 'CAYENNE', '3008', 'CLA-OSZTÁLY', '5', 'XC90', 'SORENTO', '4-ES SOROZAT', 'PRIUS', 'TRANSPORTER', 'PANDA', 'TWINGO', 'RIO', '159', 'ECOSPORT', 'AYGO', '208', 'MOKKA', 'S60']
data['Modellcsoport modellezéshez'] = data['Modellcsoport modellezéshez'].apply(lambda x: 'EGYÉB' if x not in modellcsoport_count else x)

data['Modellcsoport modellezéshez'] = data['Márka'] + ' ' + data['Modellcsoport modellezéshez']
data.loc[data['Modellcsoport modellezéshez'] == 'MERCEDES-BENZ 500', 'Modellcsoport modellezéshez'] = 'MERCEDES-BENZ EGYÉB'
data.loc[data['Modellcsoport modellezéshez'] == 'TOYOTA PRIUS', 'Modellcsoport modellezéshez'] = 'TOYOTA EGYÉB'

Wall time: 200 ms


#### Expand felszereltség dictionaty into 4 columns

In [18]:
%%time

data['Felszereltség'] = data['Felszereltség'].apply(lambda x: literal_eval(x) if isinstance(x, float) == False else {})
felszereltseg_expanded = data['Felszereltség'].apply(pd.Series)
data = pd.concat([data, felszereltseg_expanded], 1)
data.drop('Felszereltség', 1, inplace = True)

Wall time: 40.6 s


After cleaning and normalizing categories

In [19]:
%%time

data[['Állapot', 'Kivitel', 'Szállítható szem. száma', 'Ajtók száma', 'Üzemanyag', 'Hajtás', 'Sebességváltó fajtája', 'Okmányok jellege', 'URL', 'Hirdetés név', 'Kategória', 'Márka', 'Modellcsoport', 'Modell', 'Szín', 'Klíma fajtája', 'Műszaki vizsga érvényes', 'Modellcsoport modellezéshez']].describe().T

Wall time: 382 ms


,count,unique,top,freq
Állapot,72546,6,Normál,26300
Kivitel,72546,8,Ferdehátú,23182
Szállítható szem. száma,72546,4,=5,59766
Ajtók száma,72546,5,5.0,52072
Üzemanyag,72183,4,Benzin,38699
Hajtás,67767,3,Első kerék,50105
Sebességváltó fajtája,68509,2,Manuális,48442
Okmányok jellege,72538,4,Érvényes,63522
URL,72546,72546,https://www.hasznaltauto.hu/szemelyauto/suzuki...,1
Hirdetés név,72546,53622,BMW 320d,76


### Check numeric features - distributions, correlations

In [20]:
data.select_dtypes(exclude = 'O').describe(percentiles = [.01, .1, .25, .5, .75, .9, .99]).T.astype(int)

,count,mean,std,min,1%,10%,25%,50%,75%,90%,99%,max
Extrákkal növelt ár,6956,11608123,7090438,3799000,4460000,5890000,7286000,9174800,13360725,20586485,38402926,78905000
Akciós ár,6488,9642676,5953084,220000,1799000,5050000,6140000,7855000,10990000,17000000,33107550,66090000
Vételár EUR,72536,14138,19853,55,491,1703,3329,6934,16615,33471,101492,193304
Évjárat,72546,2010,7,1918,1989,2002,2006,2011,2017,2021,2021,2021
Kilométeróra állása,70493,156084,141535,1,1,6000,76000,155900,221140,289706,427115,16777215
Saját tömeg,66233,1409,332,95,810,1030,1183,1370,1585,1845,2373,12340
Teljes tömeg,66015,1952,430,10,1250,1495,1690,1897,2150,2500,3160,31000
Csomagtartó,62382,444,352,1,139,250,327,435,514,586,1030,15500
Hengerűrtartalom,70349,1847,739,1,996,1199,1390,1598,1997,2967,4520,65535
Szélesség,65262,1786,84,1395,1595,1684,1735,1790,1840,1885,2008,2470


In [21]:
%%time

data.loc[data['URL'] == 'https://www.hasznaltauto.hu/szemelyauto/ford/focus/ford_focus_1.4_ambiente-17499725', 'Teljesítmény LE'] = 80
data.loc[data['URL'] == 'https://www.hasznaltauto.hu/szemelyauto/ford/focus/ford_focus_1.4_ambiente-17499725', 'Teljesítmény kW'] = 59
data.loc[data['URL'] == 'https://www.hasznaltauto.hu/szemelyauto/hyundai/i20/hyundai_i20_1.25_life_ac_119.000_kmvegig_vezetett_szerzikonyv-17502455', 'Teljesítmény LE'] = 84
data.loc[data['URL'] == 'https://www.hasznaltauto.hu/szemelyauto/hyundai/i20/hyundai_i20_1.25_life_ac_119.000_kmvegig_vezetett_szerzikonyv-17502455', 'Teljesítmény kW'] = 62
data.loc[data['URL'] == 'https://www.hasznaltauto.hu/szemelyauto/suzuki/across/suzuki_across-17483570', 'Teljesítmény LE'] = 306
data.loc[data['URL'] == 'https://www.hasznaltauto.hu/szemelyauto/suzuki/across/suzuki_across-17483570', 'Teljesítmény kW'] = 225
data.loc[data['URL'] == 'https://www.hasznaltauto.hu/szemelyauto/opel/corsa_c/opel_corsa_c-17431135', 'Teljesítmény LE'] = 70
data.loc[data['URL'] == 'https://www.hasznaltauto.hu/szemelyauto/opel/corsa_c/opel_corsa_c-17431135', 'Teljesítmény kW'] = 52

data['Saját tömeg'] = data['Saját tömeg'].apply(lambda x: x / 1000 if x > 5000 else x)
data['Teljes tömeg'] = data['Teljes tömeg'].apply(lambda x: x / 1000 if x > 5000 else x)

data = data[data['Üzemanyagtank'] <= 120]
data = data[data['Hengerűrtartalom'] < 10000]
data = data[data['Kilométeróra állása'] < 1000000]

Wall time: 233 ms


After cleaning some extremes or mistakes

In [22]:
data.select_dtypes(exclude = 'O').describe(percentiles = [.01, .1, .25, .5, .75, .9, .99]).T.astype(int)

,count,mean,std,min,1%,10%,25%,50%,75%,90%,99%,max
Extrákkal növelt ár,3579,11170813,6730640,3865000,4410000,5828000,7153460,8710000,12915280,19700000,36958512,78905000
Akciós ár,3412,9217937,5828352,220000,1255390,4985400,5978200,7390000,10450000,16190000,32787899,57990000
Vételár EUR,59783,10810,14489,55,485,1650,3051,5871,12757,23962,74614,191390
Évjárat,59790,2010,6,1975,1996,2002,2006,2010,2015,2019,2021,2021
Kilométeróra állása,59790,166071,100111,1,1,19000,99650,166000,227500,292000,425000,990000
Saját tömeg,58818,1402,324,12,810,1029,1180,1360,1579,1840,2348,3500
Teljes tömeg,58708,1945,391,10,1250,1495,1685,1890,2140,2495,3140,4430
Csomagtartó,56764,444,355,1,139,250,325,435,513,585,1011,15500
Hengerűrtartalom,59790,1819,652,4,996,1199,1390,1598,1995,2925,4395,7011
Szélesség,59763,1784,85,1460,1593,1682,1732,1785,1839,1884,2004,2470


### Delete / consolidate features

Teljesítmény

In [23]:
data[['Teljesítmény kW', 'Teljesítmény LE']].corr()

,Teljesítmény kW,Teljesítmény LE
Teljesítmény kW,1.000000,0.999992
Teljesítmény LE,0.999992,1.000000


In [24]:
(data['Teljesítmény LE'] / data['Teljesítmény kW']).round(2).value_counts()

1.36    53908
1.35     4131
1.37     1658
1.34       11
dtype: int64

In [25]:
data[['Teljesítmény kW', 'Teljesítmény LE']].isnull().sum()

Teljesítmény kW    82
Teljesítmény LE    82
dtype: int64

Dropping kW as LE (HP) is more known for cars

In [26]:
data.drop(['Teljesítmény kW'], 1, inplace = True)

Árak

In [27]:
prices = ['Extrákkal növelt ár', 'Akciós ár', 'Vételár EUR',  'Vételár', 'Alaptípus ára']
data[prices].isnull().sum() / data.shape[0]

Extrákkal növelt ár    0.940140
Akciós ár              0.942934
Vételár EUR            0.000117
Vételár                0.061164
Alaptípus ára          0.954541
dtype: float64

In [28]:
data[prices].corr()

,Extrákkal növelt ár,Akciós ár,Vételár EUR,Vételár,Alaptípus ára
Extrákkal növelt ár,1.000000,0.959253,0.964212,NaN,0.956715
Akciós ár,0.959253,1.000000,0.999232,0.997950,0.948182
Vételár EUR,0.964212,0.999232,1.000000,0.999788,0.949477
Vételár,NaN,0.997950,0.999788,1.000000,NaN
Alaptípus ára,0.956715,0.948182,0.949477,NaN,1.000000


In [29]:
data['Vételár'] = data['Vételár'].fillna(data['Alaptípus ára'])

In [30]:
(data['Vételár'] / data['Vételár EUR']).round(0).value_counts()

361.0    40188
360.0     9725
343.0     1264
350.0      805
357.0      606
         ...  
547.0        1
606.0        1
477.0        1
517.0        1
484.0        1
Length: 369, dtype: int64

In [31]:
data['Vételár'] = data['Vételár'].fillna(data['Vételár EUR'] * 361)

In [32]:
data = data[data['Vételár'].notnull()]

In [33]:
data[prices].isnull().sum() / data.shape[0]

Extrákkal növelt ár    0.940133
Akciós ár              0.942927
Vételár EUR            0.000000
Vételár                0.000000
Alaptípus ára          0.954536
dtype: float64

In [34]:
data.drop(['Alaptípus ára', 'Vételár EUR', 'Extrákkal növelt ár'], 1, inplace = True)

Tömeg

In [35]:
data[['Saját tömeg', 'Teljes tömeg']].corr()

,Saját tömeg,Teljes tömeg
Saját tömeg,1.000000,0.965722
Teljes tömeg,0.965722,1.000000


In [36]:
data[['Saját tömeg', 'Teljes tömeg']].isnull().sum()

Saját tömeg      972
Teljes tömeg    1082
dtype: int64

In [37]:
data.drop(['Teljes tömeg'], 1, inplace = True)

Fogyasztások

In [38]:
consumption = data.filter(like = 'fogyasztás').columns.tolist()
data[consumption].corr()

,Városi fogyasztás,Országúti fogyasztás,Vegyes fogyasztás
Városi fogyasztás,1.000000,0.908487,0.976526
Országúti fogyasztás,0.908487,1.000000,0.964588
Vegyes fogyasztás,0.976526,0.964588,1.000000


In [39]:
data[consumption].isnull().sum()

Városi fogyasztás       5418
Országúti fogyasztás    5428
Vegyes fogyasztás       2283
dtype: int64

In [40]:
(data['Városi fogyasztás'] / data['Vegyes fogyasztás']).round(1).value_counts().head(3)

1.3    23697
1.2    15519
1.4     9279
dtype: int64

In [41]:
(data['Országúti fogyasztás'] / data['Vegyes fogyasztás']).round(1).value_counts().head(3)

0.8    30130
0.9    21064
0.7     1463
dtype: int64

- Városi = Vegyes * 1.2
- Országúti = Vegyes * 0.85

In [42]:
data.drop(['Országúti fogyasztás', 'Városi fogyasztás'], 1, inplace = True)

Tengelytáv & Hosszúság

In [43]:
data[['Tengelytáv', 'Hosszúság']].corr()

,Tengelytáv,Hosszúság
Tengelytáv,1.000000,0.897783
Hosszúság,0.897783,1.000000


In [44]:
data[['Tengelytáv', 'Hosszúság']].isnull().sum()

Tengelytáv     2
Hosszúság     26
dtype: int64

In [45]:
data[['Tengelytáv', 'Hosszúság']].std() / data[['Tengelytáv', 'Hosszúság']].mean()

Tengelytáv    0.065759
Hosszúság     0.086198
dtype: float64

In [46]:
data.drop('Tengelytáv', 1, inplace = True)

### All cleaning in 1 function

In [52]:
def cleaner(data):
    
    data.columns = [i.replace(':', '').strip() for i in data.columns]
    
    drop_due_to_NA = ['Oktánszám', 'Fordulási kör átmérője', 'Hasmagasság', 'Gázlómélység', 'Garancia',
                  'Átvehető', 'Első terepszög', 'Hátsó terepszög', 'Rámpaszög', 'Akció feltételei']
    for i in drop_due_to_NA:
        if i in data.columns:
            data.drop(i, 1, inplace = True)

    data = data[data['Vételár'] != 'Ár nélkül']
    data.loc[data['Vételár'].fillna('').str.contains('€'), 'Vételár'] = np.nan
    
    data['Gyorsulás 0-ról 100 km/h-ra'] = data['Gyorsulás 0-ról 100 km/h-ra'].str.replace('s', '').str.strip().str.replace(',', '.').astype(float)
    
    numeric_cols = ['Vételár', 'Vételár EUR', 'Extrákkal növelt ár', 'Akciós ár', 'Alaptípus ára',
                    'Kilométeróra állása', 'Szállítható szem. száma', 'Ajtók száma', 'Saját tömeg', 
                    'Teljes tömeg', 'Csomagtartó', 'Hengerűrtartalom', 'Szélesség', 'Magasság', 'Hosszúság', 'Tengelytáv', 
                    'Üzemanyagtank', 'Hasznos teher', 'CO2-kibocsátás', 'Végsebesség']
    for i in numeric_cols:
        if i in data.columns:
            if is_numeric_dtype(data[i]) == False:
                data[i] = data[i].str.replace(r'\D+', '').replace('', np.nan).astype(float)
                
    data[['Teljesítmény kW', 'Teljesítmény LE']] = data['Teljesítmény'].str.split(',', expand = True)
    data.drop('Teljesítmény', 1, inplace = True)
    teljesitmeny = ['Teljesítmény kW', 'Teljesítmény LE']
    for i in teljesitmeny:
        if i in data.columns:
            if is_numeric_dtype(data[i]) == False:
                data[i] = data[i].str.replace(r'\D+', '').replace('', np.nan).astype(float)
    
    fogyasztasok = data.filter(like = 'fogyasztás').columns

    for i in fogyasztasok:
        if i in data.columns:
            data[i] = data[i].str.replace('l/100km', '').str.strip().str.replace(',', '.').astype(float)
            
    data['Évjárat'] = data['Évjárat'].str.replace(r'\(.*\)', '').str.replace('(/).*', '').str.strip().astype(int)
    data['Műszaki vizsga érvényes'] = data['Műszaki vizsga érvényes'].str.replace('(/).*', '').replace('', np.nan).str.strip().astype(float)
    
    # categorical corrections
    
    data.loc[data['Állapot'].str.lower().str.contains('sérült', na = False), 'Állapot'] = 'Sérült'
    data.loc[data['Állapot'].str.lower().str.contains('hibás', na = False), 'Állapot'] = 'Sérült'
    data.loc[data['Állapot'].str.lower().str.contains('hiány', na = False), 'Állapot'] = 'Sérült'

    data = data[~data['Kivitel'].isin(['Buggy', 'Mopedautó', 'Egyéb', 'Sport'])]
    data.loc[data['Kivitel'].str.lower().str.contains('terep', na = False), 'Kivitel'] = 'Terepjáró'
    data.loc[data['Kivitel'].str.lower().str.contains('pickup', na = False), 'Kivitel'] = 'Terepjáró'
    data.loc[data['Kivitel'].str.lower().str.contains('lépcsőshátú', na = False), 'Kivitel'] = 'Coupe'

    data = data[~data['Üzemanyag'].isin(['Etanol', 'CNG', 'LPG'])]
    data.loc[data['Üzemanyag'].str.lower().str.contains('hibrid', na = False), 'Üzemanyag'] = 'Hibrid'
    data.loc[data['Üzemanyag'].str.lower().str.contains('benzin', na = False), 'Üzemanyag'] = 'Benzin'

    data.loc[data['Hajtás'].str.lower().str.contains('össz', na = False), 'Hajtás'] = 'Összkerék'

    data.loc[data['Okmányok jellege'].str.lower().str.contains('érvényes', na = False), 'Okmányok jellege'] = 'Érvényes'
    data.loc[data['Okmányok jellege'].str.lower().str.contains('lejárt', na = False), 'Okmányok jellege'] = 'Lejárt'
    data.loc[data['Okmányok jellege'].str.lower().str.contains('kivont', na = False), 'Okmányok jellege'] = 'Kivont'

    data = data[data['Klíma fajtája'] != 'Hőszivattyús klíma']
    data.loc[data['Klíma fajtája'].str.lower().str.contains('digitális', na = False), 'Klíma fajtája'] = 'Digitális'
    data.loc[data['Klíma fajtája'].str.lower().str.contains('manuális', na = False), 'Klíma fajtája'] = 'Manuális'
    data.loc[data['Klíma fajtája'].str.lower().str.contains('automata', na = False), 'Klíma fajtája'] = 'Automata'

    data['Szín - metál'] = data['Szín'].apply(lambda x: 1 if 'metál' in str(x).lower() else 0)
    data['Szín'] = data['Szín'].str.replace(r'\(.*\)', '').str.strip().replace('', np.nan)
    data.loc[data['Szín'].str.lower().str.contains('szürke', na = False), 'Szín'] = 'Szürke'
    data.loc[data['Szín'].str.lower().str.contains('kék', na = False), 'Szín'] = 'Kék'
    data.loc[data['Szín'].str.lower().str.contains('narancs', na = False), 'Szín'] = 'Narancssárga'
    data.loc[data['Szín'].str.lower().str.contains('sárga', na = False), 'Szín'] = 'Sárga'
    data.loc[data['Szín'].str.lower().str.contains('okker', na = False), 'Szín'] = 'Sárga'
    data.loc[data['Szín'].str.lower().str.contains('fehér', na = False), 'Szín'] = 'Fehér'
    data.loc[data['Szín'].str.lower().str.contains('barna', na = False), 'Szín'] = 'Barna'
    data.loc[data['Szín'].str.lower().str.contains('zöld', na = False), 'Szín'] = 'Zöld'
    data.loc[data['Szín'].str.lower().str.contains('türkiz', na = False), 'Szín'] = 'Kék'
    top_szinek = ['Fekete', 'Szürke', 'Fehér', 'Ezüst', 'Kék', 'Piros', 'Barna', 'Zöld', 'Bordó', 'Sárga']
    data.loc[~data['Szín'].isin(top_szinek) & data['Szín'].notnull(), 'Szín'] = 'Egyéb'

    data.loc[data['Sebességváltó fajtája'].str.lower().str.contains('automata', na = False), 'Sebességváltó fajtája'] = 'Automata'
    data.loc[data['Sebességváltó fajtája'].str.lower().str.contains('szekvenciális', na = False), 'Sebességváltó fajtája'] = 'Automata'
    data.loc[data['Sebességváltó fajtája'].str.lower().str.contains('manuális', na = False), 'Sebességváltó fajtája'] = 'Manuális'
    data.loc[data['Sebességváltó fajtája'].str.lower().str.contains('tiptronic', na = False), 'Sebességváltó fajtája'] = 'Automata'

    drop_brands = ['SMART', 'JAGUAR', 'LADA', 'CHRYSLER', 'MERCEDES-AMG', 'SSANGYONG', 'DAEWOO', 'SUBARU', 'TESLA', 'LANCIA', 'DODGE', 'INFINITI', 'SAAB', 'DAIHATSU', 'TRABANT', 'MASERATI', 'ROVER', 'CADILLAC', 'ABARTH', 'WARTBURG', 'CUPRA', 'LINCOLN', 'FERRARI', 'POLSKI FIAT', 'BENTLEY', 'VERSENYAUTÓ', 'ISUZU', 'MG', 'PONTIAC', 'HUMMER', 'ASTON MARTIN', 'GAZ', 'UAZ', 'ROLLS-ROYCE', 'MARUTI', 'DS', 'BUICK', 'TATA', 'BLUECAR', 'GMC', 'ZASTAVA', 'VOLGA', 'MOSZKVICS', 'IVECO', 'ACURA', 'OLDSMOBILE', 'ALPINA', 'MAHINDRA', 'LAMBORGHINI', 'FORD (USA)', 'NSU', 'MATRA', 'DATSUN', 'FISKER', 'YUGO', 'LOTUS', 'BARKAS', 'POLESTAR', 'AIXAM', 'GAC GONOW', 'GREAT WALL', 'DAIMLER', 'INNOCENTI', 'PROTON', 'DKW', 'PLYMOUTH', 'LUAZ', 'AUTOBIANCHI', 'TAZZARI', 'DUTTON', 'ARO', 'BERTONE', 'MERCURY', 'WARSZAWA', 'BRILLIANCE (BMW)', 'ADLER', 'WOLSELEY', 'ASIA', 'REPLIKA', 'POLARIS', 'TRIUMPH', 'HUDSON', 'FORD (AUSTRALIA)', 'PACKARD', 'GEO', 'NASH', 'MORGAN', 'SVW (VOLKSWAGEN)', 'ZAPOROZSEC', 'LIGIER', 'EXCALIBUR', 'EGYEDI', 'MERCEDES-MAYBACH', 'GOGGOMOBIL', 'BORGWARD', 'TALBOT', 'BRILLIANCE (ZHONGHUA)', 'AUSTIN']
    data = data[~data['Márka'].isin(drop_brands)]
    
    modellcsoport_used = ['AUDI', 'BMW', 'CITROEN', 'FIAT', 'FORD', 'HYUNDAI', 'JEEP', 'KIA', 'MAZDA', 'MERCEDES-BENZ', 'MINI', 'MITSUBISHI', 'NISSAN', 'OPEL', 'RENAULT', 'SUZUKI', 'TOYOTA', 'VOLKSWAGEN', 'VOLVO']
    data.loc[(data['Márka'].isin(modellcsoport_used)) & (data['Modellcsoport'].isnull()), 'Modellcsoport'] = data.loc[(data['Márka'].isin(modellcsoport_used)) & (data['Modellcsoport'].isnull()), 'Modell']

    # some nan rules
    
    data['Műszaki vizsga érvényes'] = data['Műszaki vizsga érvényes'].apply(lambda x: 'Érvényes' if x >= 2021 else 'Lejárt / Nincs adat')

    data['Ajtók száma'].fillna('Nincs adat', inplace = True)
    data['Ajtók száma'] = data['Ajtók száma'].astype(str)

    data = data[(data['Szállítható szem. száma'] < 20) | (data['Szállítható szem. száma'].isnull())]
    data['Szállítható szem. száma'] = data['Szállítható szem. száma'].apply(lambda x: '<5' if x < 5 else ('>5' if x > 5 else ('=5' if x == 5 else 'NA')))
    
    # modellcsoport modellezeshez
    
    data['Modellcsoport modellezéshez'] = data['Modellcsoport'].copy()

    modell_count = ['FOCUS', 'OCTAVIA', 'A4', 'SWIFT', 'A6', 'MEGANE', 'MONDEO', 'ASTRA H', 'FIESTA', '320', "CEE'D", 'YARIS', 'A3', 'CLIO', 'FABIA', 'I30', 'CIVIC', '6', 'ASTRA J', 'QASHQAI', '500', 'COROLLA', 'PASSAT', 'CORSA D', 'ASTRA G', '3', 'INSIGNIA', 'X5', '530', 'SUPERB', 'KUGA', 'JAZZ', 'C-MAX', 'POLO', 'ASTRA K', 'AVENSIS', '206', '318', 'GOLF VII', 'VITARA', 'PASSAT VIII', 'PUNTO', '520', 'AURIS', 'GOLF V', 'LEON', 'PUMA', 'SCENIC', 'X3', 'TRANSIT', 'IBIZA', 'E 220', 'MICRA', 'IGNIS', 'GOLF VI', 'SPORTAGE', '308', 'TOURAN', 'SX4', 'TIPO', 'C3', 'CORSA F', 'X-TRAIL', 'A5', 'C 220', 'Q7', 'JUKE', 'RAV 4', 'S-MAX', 'ZAFIRA B', 'LAGUNA', 'X6', '307', 'GOLF IV', 'TIGUAN', '207', 'I3', '330', '2', 'MERIVA A', 'DUSTER', 'XC60', 'GALAXY', 'I20', 'Q5', 'ACCORD', '525', 'GOLF', 'LEAF', 'XSARA PICASSO', 'TUCSON', 'C4', 'SANDERO', 'MERIVA', '116', 'C 200', 'CORSA C', 'CROSSLAND', 'MUSTANG', 'CR-V', 'SX4 S-CROSS', 'TOUAREG', 'C5', 'OUTLANDER', 'GRANDE PUNTO', 'ALTEA', 'PASSAT VI', 'A8', '316', 'COLT', '5008', '3008', 'E 200', 'CAYENNE', '5', 'XC90', 'SORENTO', 'ASTRA', 'GOLF PLUS', 'ASTRA F', 'PANDA', 'TWINGO', 'RIO', '3-AS SOROZAT', '159', 'VECTRA C', 'ECOSPORT', '208', 'AYGO', 'ESPACE', 'MOKKA', 'PASSAT V', 'X1', 'S60', 'CORSA', 'C 180', 'POLO IV']
    data.loc[(data['Modellcsoport'].isnull()) & (data['Modell'].isin(modell_count)), 'Modellcsoport modellezéshez'] = data.loc[(data['Modellcsoport'].isnull()) & (data['Modell'].isin(modell_count)), 'Modell']

    modellcsoport_count = ['ASTRA', '3-AS SOROZAT', 'FOCUS', 'GOLF', 'PASSAT', 'X SOROZAT', '5-ÖS SOROZAT', 'OCTAVIA', 'CORSA', 'E-OSZTÁLY', 'A4', 'C-OSZTÁLY', 'SWIFT', 'A6', 'MEGANE', 'MONDEO', "CEE'D", 'FIESTA', 'POLO', 'YARIS', 'A3', 'COROLLA', 'CLIO', 'FABIA', 'ZAFIRA', 'I30', 'MERIVA', 'A-OSZTÁLY', 'CIVIC', '6', 'QASHQAI', 'PUNTO', '1-ES SOROZAT', '500', '3', 'SCENIC', 'INSIGNIA', 'SUPERB', 'KUGA', 'JAZZ', 'AVENSIS', 'VITARA', 'S-OSZTÁLY', 'C-MAX', '206', '7-ES SOROZAT', 'AURIS', 'LEON', 'PUMA', 'B-OSZTÁLY', 'MICRA', 'TRANSIT', 'IBIZA', 'A5', 'C4 PICASSO', 'IGNIS', 'SPORTAGE', 'XSARA', '308', 'TOURAN', 'SX4', 'TIPO', 'VECTRA', 'C3', 'X-TRAIL', 'I SOROZAT', 'Q7', 'JUKE', 'TIGUAN', 'RAV 4', 'S-MAX', 'LAGUNA', 'MINI', 'JETTA', '307', '207', '2', 'DUSTER', 'XC60', 'I20', 'GALAXY', 'ACCORD', 'Q5', 'ML-OSZTÁLY', 'LEAF', 'GLE-OSZTÁLY', 'TUCSON', 'SANDERO', 'C4', 'CROSSLAND', 'MUSTANG', 'CR-V', 'SX4 S-CROSS', 'ESPACE', 'TOUAREG', 'C5', 'OUTLANDER', 'ALTEA', 'A8', 'GLC-OSZTÁLY', 'M SOROZAT', 'COLT', '5008', 'CAYENNE', '3008', 'CLA-OSZTÁLY', '5', 'XC90', 'SORENTO', '4-ES SOROZAT', 'PRIUS', 'TRANSPORTER', 'PANDA', 'TWINGO', 'RIO', '159', 'ECOSPORT', 'AYGO', '208', 'MOKKA', 'S60']
    data['Modellcsoport modellezéshez'] = data['Modellcsoport modellezéshez'].apply(lambda x: 'EGYÉB' if x not in modellcsoport_count else x)

    data['Modellcsoport modellezéshez'] = data['Márka'] + ' ' + data['Modellcsoport modellezéshez']
    data.loc[data['Modellcsoport modellezéshez'] == 'MERCEDES-BENZ 500', 'Modellcsoport modellezéshez'] = 'MERCEDES-BENZ EGYÉB'
    data.loc[data['Modellcsoport modellezéshez'] == 'TOYOTA PRIUS', 'Modellcsoport modellezéshez'] = 'TOYOTA EGYÉB'
    
    data['Felszereltség'] = data['Felszereltség'].apply(lambda x: literal_eval(x) if isinstance(x, float) == False else {})
    felszereltseg_expanded = data['Felszereltség'].apply(pd.Series)
    data = pd.concat([data, felszereltseg_expanded], 1)
    data.drop('Felszereltség', 1, inplace = True)
    
    # numerical corrections
    
    data.loc[data['URL'] == 'https://www.hasznaltauto.hu/szemelyauto/ford/focus/ford_focus_1.4_ambiente-17499725', 'Teljesítmény LE'] = 80
    data.loc[data['URL'] == 'https://www.hasznaltauto.hu/szemelyauto/ford/focus/ford_focus_1.4_ambiente-17499725', 'Teljesítmény kW'] = 59
    data.loc[data['URL'] == 'https://www.hasznaltauto.hu/szemelyauto/hyundai/i20/hyundai_i20_1.25_life_ac_119.000_kmvegig_vezetett_szerzikonyv-17502455', 'Teljesítmény LE'] = 84
    data.loc[data['URL'] == 'https://www.hasznaltauto.hu/szemelyauto/hyundai/i20/hyundai_i20_1.25_life_ac_119.000_kmvegig_vezetett_szerzikonyv-17502455', 'Teljesítmény kW'] = 62
    data.loc[data['URL'] == 'https://www.hasznaltauto.hu/szemelyauto/suzuki/across/suzuki_across-17483570', 'Teljesítmény LE'] = 306
    data.loc[data['URL'] == 'https://www.hasznaltauto.hu/szemelyauto/suzuki/across/suzuki_across-17483570', 'Teljesítmény kW'] = 225
    data.loc[data['URL'] == 'https://www.hasznaltauto.hu/szemelyauto/opel/corsa_c/opel_corsa_c-17431135', 'Teljesítmény LE'] = 70
    data.loc[data['URL'] == 'https://www.hasznaltauto.hu/szemelyauto/opel/corsa_c/opel_corsa_c-17431135', 'Teljesítmény kW'] = 52

    data['Saját tömeg'] = data['Saját tömeg'].apply(lambda x: x / 1000 if x > 5000 else x)
    data['Teljes tömeg'] = data['Teljes tömeg'].apply(lambda x: x / 1000 if x > 5000 else x)

    data = data[data['Üzemanyagtank'] <= 120]
    data = data[data['Hengerűrtartalom'] < 10000]
    data = data[data['Kilométeróra állása'] < 1000000]
    
    
    # dropping / consolidating
    data.drop(['Teljesítmény kW'], 1, inplace = True)
    data['Vételár'] = data['Vételár'].fillna(data['Alaptípus ára'])
    data['Vételár'] = data['Vételár'].fillna(data['Vételár EUR'] * 361)
    data = data[data['Vételár'].notnull()]
    data.drop(['Alaptípus ára', 'Vételár EUR', 'Extrákkal növelt ár'], 1, inplace = True)
    data.drop(['Teljes tömeg'], 1, inplace = True)
    data.drop(['Országúti fogyasztás', 'Városi fogyasztás'], 1, inplace = True)
    data.drop('Tengelytáv', 1, inplace = True)
    
    return data

In [53]:
%%time

data = pd.read_csv('data/autok_final.csv.gz', compression = 'gzip')

Wall time: 7.72 s


In [54]:
%%time

print(data.shape)
data = cleaner(data)
print(data.shape)

(76896, 52)
(59783, 40)
Wall time: 56.8 s


In [55]:
data.to_csv('data/autok_final_cleaned.csv.gz', compression = 'gzip', index = False)